In [ ]:
from WealthIncomeMPModel_case import WealthIncomeModelClass_CaseStudy
import numpy as np
import sympy
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams.update({
    "axes.grid": True,
    "grid.color": "black",
    "grid.alpha": 0.25,
    "grid.linestyle": "--",
    "font.size": 14                 
})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
from EconDLSolvers import choose_gpu
import torch, gc
import os
import numpy as np
import pandas as pd
# Hyperparameters
device = choose_gpu()
model_DL = {}
torch.cuda.empty_cache()

# 1. Initialize both models 
model_DL[r'Baseline'] = WealthIncomeModelClass_CaseStudy(
    device=device,
    load='DeepSimulate_hetero_beta.pkl'
)

model_DL[r'Experiment'] = WealthIncomeModelClass_CaseStudy(
    device=device,
    load='DeepSimulate_hetero_beta.pkl'
)


# 2. modify par manually
model_DL[r'Baseline'].par.R_shock = 0
model_DL[r'Baseline'].par.experiment = None
model_DL[r'Baseline'].sim.N = 10_000

model_DL[r'Experiment'].par.experiment = "R1pp"
model_DL[r'Experiment'].par.R_shock = -0.01
#model_DL[r'Experiment'].par.shock_periods = [(10,20)]
model_DL[r'Experiment'].sim.N = 10_000




In [ ]:
def simulate_model(model):
    """ Simulate full lifecycle for one model. """
    
    par = model.par
    sim = model.sim

    # 1. Draw initial states
    initial_states = model.draw_initial_states(sim.N)
    print(f"initial_states shape: {initial_states.shape}")

    # 2. Draw shocks
    shocks = model.draw_shocks(sim.N)
    print(f"shocks shape: {shocks.shape}")

    # 3. Move shocks to same device
    device = sim.states.device
    shocks = shocks.to(device)

    # 4. Set into simulation
    sim.states[0] = initial_states
    sim.shocks = shocks

    # 5. Simulation loop
    for t in range(par.T-1):
        states_t = sim.states[t,:,:]
        shocks_t = sim.shocks[t]

        actions_t = model.eval_policy(model.policy_NN, states_t.unsqueeze(0))
        actions_t = actions_t.squeeze(0)

        outcomes_t = model.outcomes(states_t, actions_t, t0=t, t=t)
        reward_t = model.reward(states_t, actions_t, outcomes_t, t0=t)

        states_pd_t = model.state_trans_pd(states_t, actions_t, outcomes_t, t0=t)
        states_tp1 = model.state_trans(states_pd_t, shocks_t, t=t)

        sim.states[t+1] = states_tp1
        sim.outcomes[t] = outcomes_t
        sim.actions[t] = actions_t
        sim.reward[t] = reward_t
    return sim



In [ ]:
def simulate_model(model):
    par = model.par
    sim = model.sim
    N   = sim.N           

    # ────────────────────────────────
    # 0) Trim every sim buffer to the new N
    # ────────────────────────────────
    # states[t] has shape (old_N, n_states) – we want (N, n_states)
    sim.states   = sim.states  [:, :N, :].contiguous()
    sim.actions  = sim.actions [:, :N, :].contiguous()
    sim.outcomes = sim.outcomes[:, :N, :].contiguous()
    sim.reward   = sim.reward  [:, :N].contiguous()
    
    if hasattr(sim, "shocks"):
        sim.shocks = sim.shocks[:, :N, :].contiguous()

    # ────────────────────────────────
    # 1) Draw initial states & shocks
    # ────────────────────────────────
    device         = sim.states.device
    initial_states = model.draw_initial_states(N).to(device)  # -> (N, 8)
    shocks         = model.draw_shocks(N).to(device)          # -> (T, N, n_shocks)

    # ────────────────────────────────
    # 2) Seed the sim with period-0
    # ────────────────────────────────
    sim.states[0] = initial_states    # now (N,8) ← (N,8)
    sim.shocks     = shocks

    # ────────────────────────────────
    # 3) Run the rest of the simulation
    # ────────────────────────────────
    for t in range(par.T-1):
        states_t   = sim.states[t]           # (N,8)
        shocks_t   = sim.shocks[t]           # (N,n_shocks)
        actions_t  = model.eval_policy(
                          model.policy_NN,
                          states_t.unsqueeze(0)
                     ).squeeze(0)           # (N,n_actions)
        outcomes_t = model.outcomes(states_t, actions_t, t0=t, t=t)
        reward_t   = model.reward(states_t, actions_t, outcomes_t, t0=t)
        pd_t       = model.state_trans_pd(states_t, actions_t, outcomes_t, t0=t)
        next_s     = model.state_trans(pd_t, shocks_t, t=t)

        sim.states [t+1] = next_s
        sim.outcomes[t]  = outcomes_t
        sim.actions [t]  = actions_t
        sim.reward  [t]  = reward_t

    return sim


In [ ]:
# Loop over all models in model_DL
for model_name, model in model_DL.items():
    print(f"Simulating {model_name}...")
    simulate_model(model)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# --------------------------------------------------
# 1.  CREATE CANVAS   (2 rows × 3 columns = 6 slots)
# --------------------------------------------------
fig, axes = plt.subplots(
    nrows=2, ncols=3, figsize=(16, 6),
    constrained_layout=True
)
axes = axes.flatten()

titles = [
    'Consumption',     # 0
    'Labor Supply',    # 1
    'Resources',       # 2  <-- fixed spelling
    'Liquid Asset',    # 3
    'Illiquid Asset',  # 4
    'New Debt'         # 5
]

var_to_ax = {i: i for i in range(6)}

# --------------------------------------------------
# 2.  PLOT THE SERIES
# --------------------------------------------------
for model_key, model_obj in model_DL.items():
    outcomes = model_obj.sim.outcomes.detach().cpu().numpy()
    age = np.arange(outcomes.shape[0]) + 18

    for var_idx, ax_idx in var_to_ax.items():
        data = outcomes[:, :, var_idx]
        mean = data.mean(axis=1)
        std  = data.std(axis=1)
        p5, p95 = np.percentile(data, [5, 95], axis=1)

        ax = axes[ax_idx]
        ax.plot(age, mean, lw=2, label=model_key)
        lower = np.maximum(mean - 2*std, 0)
        upper = mean + 2*std
        ax.fill_between(age, lower, upper, alpha=0.25)       # <-- use age here
        ax.plot(age, p5,  ls='--', alpha=0.5)
        ax.plot(age, p95, ls='--', alpha=0.5)
        ax.axvline(67, ls=':', color='k', alpha=0.7)

# --------------------------------------------------
# 3.  AESTHETICS
# --------------------------------------------------
for idx, title in enumerate(titles):
    axes[idx].set_title(title)

for ax in axes:
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

# --------------------------------------------------
# 4.  SINGLE, DEDUPLICATED LEGEND
# --------------------------------------------------
#handles, labels = axes[0].get_legend_handles_labels()
#uniq = {lab: h for h, lab in zip(handles, labels)}

handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels,
           loc='lower center', bbox_to_anchor=(0.5, -0.1),
           ncol=len(labels), frameon=False)

# 5) save, using bbox_inches='tight' to grab the legend too
fig.savefig(
    "Outcomes_Baseline_Experiment_SOFT.png",
    dpi=300,
    bbox_inches="tight"
)
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# --- 1) Define shares to plot ---
# 1: illiquid, 2: liquid, 3: debt; plus computed consumption
plot_order = [
    (1, 'Illiquid Share'),
    (2, 'Liquid Share'),
    (3, 'Debt Share'),
    ('cons', 'Consumption Share')
]

age_start = 18
ret_age   = 67

# --- 2) Prepare 2×2 grid ---
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex=True)
axes = axes.flatten()

# --- 3) Loop over models and panels ---
for model_key in model_DL:
    actions = model_DL[model_key].sim.actions.detach().cpu().numpy()
    T       = actions.shape[0]
    x       = np.arange(age_start, age_start + T)

    # compute consumption share
    ill  = actions[:, :, 1]
    liq  = actions[:, :, 2]
    cons = (1 - ill) * (1 - liq)

    for ax, (v_idx, title) in zip(axes, plot_order):
        if v_idx == 'cons':
            data = cons
            xplt = x
        else:
            data = actions[:, :, v_idx]
            if title == 'Debt Share':
                mask = x <= ret_age
                xplt = x[mask]
                data = data[mask, :]
            else:
                xplt = x

        mean    = data.mean(1)
        std     = data.std(1)
        p5, p95 = np.percentile(data, [5, 95], axis=1)

        lower = np.maximum(mean - 2*std, 0)
        upper = np.minimum(mean + 2*std, 1)
        p5    = np.maximum(p5, 0)
        p95   = np.minimum(p95, 1)

        ax.plot(xplt, mean, lw=2, label=model_key)
        ax.fill_between(xplt, lower, upper, alpha=0.2)
        ax.plot(xplt, p5 , ls='--', alpha=0.5)
        ax.plot(xplt, p95, ls='--', alpha=0.5)

        # retirement line on all panels
        ax.axvline(ret_age, color='black', ls=':', lw=1)

        ax.set_title(title)
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        ax.set_ylim(0,1)

# --- 4) Configure x-axis only on bottom row ---
for idx, ax in enumerate(axes):
    if idx >= 2:
        ax.set_xlabel("Age")
    else:
        ax.tick_params(labelbottom=False)

# --- 5) Legend & layout ---
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels,
           loc='lower center', bbox_to_anchor=(0.5, 0.02),
           ncol=len(labels), frameon=False)

fig.tight_layout(rect=[0, 0.05, 1, 0.95])

fig.savefig('Actions_Baseline_Experiment_SOFT.png', dpi=300)
#fig.savefig('Actions_Baseline_Experiment.png', dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math

# --- 1) Which state‐indices to show (no Illiquid Assets idx=0) ---
core_vars = {
    1: 'Wage',
    2: 'Cash-on-Hand',
    3: 'Inflation',
    4: 'Nominal Interest Rate',
    5: 'Return on Illiquid Assets',
    6: 'Debt'
}

age_start = 18
n_vars    = len(core_vars)
n_cols    = 3
n_rows    = math.ceil(n_vars / n_cols)

fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(5*n_cols, 4*n_rows),
                         sharex=True)
axes = axes.flatten()

for model_key in model_DL:
    states = model_DL[model_key].sim.states.detach().cpu().numpy()
    T      = states.shape[0]
    x      = np.arange(age_start, age_start + T)

    for pos, (var_idx, title) in enumerate(core_vars.items()):
        ax   = axes[pos]
        data = states[:, :, var_idx]     # (T, n_sims)

        mean    = data.mean(1)
        std     = data.std(1)
        p5, p95 = np.percentile(data, [5,95], axis=1)

        ax.plot(x, mean, lw=2, label=model_key)
        lower = np.maximum(mean - 2*std, 0)
        upper = mean+2*std
        ax.fill_between(x, lower, upper, alpha=0.25)
        ax.plot(x, p5 , ls='--', alpha=0.5)
        ax.plot(x, p95, ls='--', alpha=0.5)
        ax.axvline(age_start + 49, color='gray', ls=':', lw=1)
        ax.set_title(title)
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

# hide unused panels
for extra_ax in axes[n_vars:]:
    extra_ax.axis('off')

# show x‐ticklabels only on bottom row
for pos, ax in enumerate(axes[:n_vars]):
    if pos // n_cols == n_rows - 1:
        ax.set_xlabel("Age")
        ax.tick_params(labelbottom=True)
    else:
        ax.tick_params(labelbottom=False)

# legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels,
           loc='lower center', bbox_to_anchor=(0.5, 0.02),
           ncol=len(labels), frameon=False)

fig.tight_layout(rect=[0, 0.05, 1, 0.95])
#fig.savefig("States_Baseline_Experiment_SOFT.png", dpi=300, bbox_inches='tight')
#fig.savefig("States_DeepVPD.png", dpi=300)
#fig.savefig("States_DeepSimulate.png", dpi=300)
plt.show()




In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch


# --- 2) Net-wealth helper ---
def compute_net_wealth(states, outcomes):
    ill  = outcomes[..., 4]
    liq  = outcomes[..., 3]
    debt = states[..., 6]
    nd   = outcomes[..., 5]
    return ill + liq - debt - nd

# --- 3) Age-bin definitions ---
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
age_labels = ["18–", "35–", "50–", "65–", "75–", "85-"]

clr_hi = "#ff9999"
clr_lo = "#66b3ff"

# --- 4) Loop over bins, each with fresh copies of the models ---
bins_hi = []  # Baseline net-wealth pools
bins_lo = []  # Experiment net-wealth pools

for (a_lo, a_hi) in age_bins:
    t_lo = a_lo - age_start

    # ---- Clone both models so no state carries over ----
    model_b = copy.deepcopy(model_DL['Baseline'])
    model_e = copy.deepcopy(model_DL['Experiment'])

    # --- Baseline run: no shocks ever ---
    model_b.par.shock_periods = []
    model_b.sim.N = 10_000
    sim_b = simulate_model(model_b)
    sh_b, oh_b = sim_b.states.detach().cpu().numpy(), sim_b.outcomes.detach().cpu().numpy()
    nw_b       = compute_net_wealth(sh_b, oh_b)

    # --- Experiment run: one-off shock at t_lo only ---
    model_e.par.shock_periods = [t_lo]
    model_e.sim.N = 10_000
    sim_e = simulate_model(model_e)
    sh_e, oh_e = sim_e.states.detach().cpu().numpy(), sim_e.outcomes.detach().cpu().numpy()
    nw_e       = compute_net_wealth(sh_e, oh_e)

    # pool the *post-shock* lifecyle (from t_lo on) across all agents
    pooled_b = nw_b[t_lo:, :].ravel()
    pooled_e = nw_e[t_lo:, :].ravel()

    bins_hi.append(pooled_b)
    bins_lo.append(pooled_e)

# --- 5) Plotting the two box‐series side by side ---
labels = age_labels
x      = np.arange(len(labels))
pos_hi = x - 0.15
pos_lo = x + 0.15

common_kw = dict(
    notch=True,
    showfliers=False,
    whis=1.5,
    medianprops=dict(color="black", linewidth=1.2),
)

fig, ax = plt.subplots(figsize=(10, 6))

bp_hi = ax.boxplot(
    bins_hi, positions=pos_hi, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_hi, alpha=0.6, linewidth=1),
    **common_kw
)
bp_lo = ax.boxplot(
    bins_lo, positions=pos_lo, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_lo, alpha=0.6, linewidth=1),
    **common_kw
)

ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel("Net-Wealth Level")
ax.legend(
    [bp_hi["boxes"][0], bp_lo["boxes"][0]],
    ["Baseline", "Experiment"],
    loc="upper left",
    frameon=False
)

# Annotate one example bin (“50–64”) 
i = labels.index("50–64")
data = bins_hi[i]
q1, med, q3 = np.percentile(data, [25,50,75])
iqr         = q3 - q1
whisk_low   = np.min(data[data >= q1 - 1.5*iqr])
whisk_high  = np.max(data[data <= q3 + 1.5*iqr])
mean_val    = np.mean(data)

stat_text = (
    "Age 50–64, Baseline\n"
    f"Min    = {whisk_low:.0f}\n"
    f"Median = {med:.0f}\n"
    f"Mean   = {mean_val:.0f}\n"
    f"Max    = {whisk_high:.0f}"
)
ax.annotate(
    stat_text,
    xy=(pos_hi[i], med),
    xytext=(0.02, 0.75),
    textcoords="axes fraction",
    arrowprops=dict(arrowstyle="->", lw=1),
    va="center", ha="left",
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.4", facecolor="white", edgecolor="gray")
)

plt.tight_layout()
plt.show()
fig.savefig("boxplot_netwealth_base_experiment_oneoff.png", dpi=300)


In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch


# --- 2) Net-wealth helper ---
def compute_net_wealth(states, outcomes):
    ill  = outcomes[..., 4]
    liq  = outcomes[..., 3]
    debt = states[..., 6]
    nd   = outcomes[..., 5]
    return ill + liq - debt - nd


# --- 3) Age-bin definitions (unchanged) ---
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
age_labels = ["18–", "35–", "50–", "65–", "75–", "85-"]

torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw    = model_DL['Baseline'].draw_shocks(N).to(device)
model_DL['Baseline'].sim.shocks   = raw.clone()
model_DL['Experiment'].sim.shocks = raw.clone()


# --- 5) Run 3-shock experiment for each age bin, collect pct‐changes ---
percent_changes = []

for (a_lo, a_hi) in age_bins:
    t_lo  = a_lo - age_start
    #t_hi  = min(a_hi - age_start, model_DL['Baseline'].par.T - 1)
    #t_mid = (t_lo + t_hi) // 2
    shock_dates = [t_lo] #, t_mid, t_hi]

    # Baseline: reuse pre-drawn shocks, no policy shocks
    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    sb = simulate_model(mb)
    sb_s, sb_o = sb.states.detach().cpu().numpy(), sb.outcomes.detach().cpu().numpy()
    nw_b = compute_net_wealth(sb_s, sb_o)

    # Experiment: same shocks + policy shocks at start, mid, end
    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = shock_dates
    se = simulate_model(me)
    se_s, se_o = se.states.detach().cpu().numpy(), se.outcomes.detach().cpu().numpy()
    nw_e = compute_net_wealth(se_s, se_o)

    # percent‐change (exp vs base) over t_lo onward
    pct = (nw_e[t_lo:, :] - nw_b[t_lo:, :]) / nw_b[t_lo:, :] * 100
    percent_changes.append(pct.ravel())



# --- 6) Plot distribution of 3-shock impacts by age bin ---
fig, ax = plt.subplots(figsize=(10, 6))
xpos = np.arange(len(age_labels))

# 6a) draw boxplot with 1.5×IQR whiskers
bp = ax.boxplot(
    percent_changes,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1)
)

# 6b) color boxes
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

# 6c) zero line
ax.axhline(0, linestyle="--", color="gray", linewidth=1)

# 6d) cohort medians
cohort_medians = np.array([np.nanmedian(pc) for pc in percent_changes])
ax.scatter(
    xpos, cohort_medians,
    marker='D', s=60,
    facecolors='#2f4f4f', edgecolors='white',
    linewidths=1.2, zorder=4,
    label="Cohort Median"
)

# 6e) whisker‐restricted cohort means
cohort_means = []
for i, pc in enumerate(percent_changes):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    in_range = pc[(pc >= lo) & (pc <= hi)]
    cohort_means.append(np.nanmean(in_range))
cohort_means = np.array(cohort_means)

ax.scatter(
    xpos, cohort_means,
    marker='o', s=50,
    facecolors='white', edgecolors='#e6550d',
    linewidths=1.5, zorder=5,
    label="Cohort Mean"
)

# 6f) annotate median, mean, and whiskers
for i, label in enumerate(age_labels):
    x_off     = 0.35 if i < len(age_labels)-1 else 0.6
    med       = cohort_medians[i]
    mean_val  = cohort_means[i]
    lower_cap = bp["caps"][2*i].get_ydata()[0]
    upper_cap = bp["caps"][2*i+1].get_ydata()[0]

    # median label
    ax.text(xpos[i]+x_off, med,
            f"{med:.1f}%",
            ha="left", va="center",
            fontsize=9, fontweight="bold", color="#2f4f4f")
    # mean label just below median
    ax.text(xpos[i]+x_off, mean_val +2,
            f"{mean_val:.1f}%",
            ha="left", va="center",
            fontsize=8, color="#e6550d")
    # whisker caps
    ax.text(xpos[i]+x_off, upper_cap + 4,
            f"max {upper_cap:.1f}%",
            ha="left", va="bottom",
            fontsize=8, color="#333333")
    ax.text(xpos[i]+x_off, lower_cap - 1,
            f"min {lower_cap:.1f}%",
            ha="left", va="top",
            fontsize=8, color="#333333")

# final formatting
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Net-Wealth")
ax.grid(axis='y', linestyle=':', linewidth=0.8, alpha=0.7)
ax.legend(frameon=False, loc='upper right')

plt.tight_layout()
#plt.savefig("mp_tight_net_wealth_boxplot.png", dpi = 300)
#plt.savefig("mp_soft_net_wealth_boxplot.png", dpi = 300)
plt.show()



In [ ]:
# --- 6) Plot distribution of 3-shock impacts by age bin ---
fig, ax = plt.subplots(figsize=(10, 6))
xpos = np.arange(len(age_labels))

# 6a) draw boxplot with 1.5×IQR whiskers
bp = ax.boxplot(
    percent_changes,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1)
)

# 6b) color boxes
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

# 6c) zero line
ax.axhline(0, linestyle="--", color="gray", linewidth=1)

# 6d) cohort medians
cohort_medians = np.array([np.nanmedian(pc) for pc in percent_changes])
ax.scatter(
    xpos, cohort_medians,
    marker='D', s=60,
    facecolors='#2f4f4f', edgecolors='white',
    linewidths=1.2, zorder=4,
    label="Cohort Median"
)

# 6e) whisker‐restricted cohort means
cohort_means = []
for i, pc in enumerate(percent_changes):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    in_range = pc[(pc >= lo) & (pc <= hi)]
    cohort_means.append(np.nanmean(in_range))
cohort_means = np.array(cohort_means)

ax.scatter(
    xpos, cohort_means,
    marker='o', s=50,
    facecolors='white', edgecolors='#e6550d',
    linewidths=1.5, zorder=5,
    label="Cohort Mean"
)

# 6f) annotate median, mean, and whiskers
for i, label in enumerate(age_labels):
    x_off     = 0.35 if i < len(age_labels)-1 else 0.3
    med       = cohort_medians[i]
    mean_val  = cohort_means[i]
    lower_cap = bp["caps"][2*i].get_ydata()[0]
    upper_cap = bp["caps"][2*i+1].get_ydata()[0]

    # median label
    ax.text(xpos[i]+x_off, med + 2,
            f"{med:.1f}%",
            ha="left", va="center",
            fontsize=9, fontweight="bold", color="#2f4f4f")
    # mean label just below median
    ax.text(xpos[i]+x_off, mean_val +15,
            f"{mean_val:.1f}%",
            ha="left", va="center",
            fontsize=8, fontweight="bold", color="#e6550d")
    # whisker caps
    ax.text(xpos[i]+x_off-0.5, upper_cap + 1.75,
            f"max {upper_cap:.1f}%",
            ha="left", va="bottom",
            fontsize=8,fontweight="bold", color="#333333")
    ax.text(xpos[i]+x_off-0.5, lower_cap - 1,
            f"min {lower_cap:.1f}%",
            ha="left", va="top",
            fontsize=8, fontweight="bold", color="#333333")

# final formatting
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Net-Wealth")
ax.grid(axis='y', linestyle=':', linewidth=0.8, alpha=0.7)
ax.legend(frameon=False, loc='upper right')

plt.tight_layout()
#plt.savefig("mp_tight_net_wealth_boxplot.png", dpi = 300)
#plt.savefig("mp_soft_net_wealth_boxplot.png", dpi = 300)
plt.show()



In [ ]:
# --- 6) Plot distribution of 3-shock impacts by age bin ---
fig, ax = plt.subplots(figsize=(10, 6))
xpos = np.arange(len(age_labels))

# 6a) draw boxplot with 1.5×IQR whiskers
bp = ax.boxplot(
    percent_changes,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)

# 6b) color boxes
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

# 6c) hide the first cohort’s box elements
#i0 = 0
# box rectangle
#bp["boxes"][i0].set_visible(False)
# the first two whiskers and caps
#bp["whiskers"][2*i0  ].set_visible(False)
#bp["whiskers"][2*i0+1].set_visible(False)
#bp["caps"][2*i0      ].set_visible(False)
#bp["caps"][2*i0+1    ].set_visible(False)
# the first median line
#bp["medians"][i0].set_visible(False)

# 6d) zero line
ax.axhline(0, linestyle="--", color="gray", linewidth=1)

# 6e) cohort medians & means for all bins
cohort_medians = np.array([np.nanmedian(pc) for pc in percent_changes])
cohort_means = []
for i, pc in enumerate(percent_changes):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    in_range = pc[(pc >= lo) & (pc <= hi)]
    cohort_means.append(np.nanmean(in_range))
cohort_means = np.array(cohort_means)

# 6f) plot medians & means for **all** cohorts
ax.scatter(
    xpos, cohort_medians,
    marker='D', s=60,
    facecolors='#2f4f4f', edgecolors='white',
    linewidths=1.2, zorder=4,
    label="Cohort Median"
)
ax.scatter(
    xpos, cohort_means,
    marker='o', s=50,
    facecolors='white', edgecolors='#e6550d',
    linewidths=1.5, zorder=5,
    label="Cohort Mean"
)

# 6g) annotate:
for i in range(len(age_labels)):
    x_off = 0.35 if i < len(age_labels)-1 else 0.3
    med       = cohort_medians[i]
    mean_val  = cohort_means[i]
    # caps for annotation
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]

    if i == 100:
        # only label median & mean (no min/max)
        ax.text(
            xpos[i]+x_off, med ,
            f"{med:.1f}%",
            ha="left", va="center",
            fontsize=9, fontweight="bold", color="#2f4f4f"
        )
        ax.text(
            xpos[i]+x_off, mean_val +1 ,
            f"{mean_val:.1f}%",
            ha="left", va="center",
            fontsize=8, fontweight="bold", color="#e6550d"
        )
    else:
        # annotate min, max, median, mean as before
        ax.text(
            xpos[i]+x_off-0.5, hi + 1.75,
            f"max {hi:.1f}%",
            ha="left", va="bottom",
            fontsize=8, fontweight="bold", color="#333333"
        )
        ax.text(
            xpos[i]+x_off-0.5, lo - 1,
            f"min {lo:.1f}%",
            ha="left", va="top",
            fontsize=8, fontweight="bold", color="#333333"
        )
        ax.text(
            xpos[i]+x_off, med ,
            f"{med:.1f}%",
            ha="left", va="center",
            fontsize=9, fontweight="bold", color="#2f4f4f"
        )
        ax.text(
            xpos[i]+x_off, mean_val +1 ,
            f"{mean_val:.1f}%",
            ha="left", va="center",
            fontsize=8, fontweight="bold", color="#e6550d"
        )

# final formatting
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Net-Wealth")
ax.grid(axis='y', linestyle=':', linewidth=0.8, alpha=0.7)
ax.legend(frameon=False, loc='upper right')
ax.set_ylim([-10, 30])

plt.tight_layout()
plt.savefig("mp_soft_net_wealth_boxplot.png", dpi=300)
plt.show()


In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch

# 1) simulate_model (unchanged)
def simulate_model(model):
    par = model.par
    sim = model.sim
    sim.states[0] = model.draw_initial_states(sim.N)
    sim.shocks    = model.draw_shocks(sim.N).to(sim.states.device)
    for t in range(par.T-1):
        s_t   = sim.states[t]
        eps_t = sim.shocks[t]
        a_t        = model.eval_policy(model.policy_NN, s_t.unsqueeze(0)).squeeze(0)
        out_t      = model.outcomes(s_t, a_t, t0=t, t=t)
        sim.reward[t] = model.reward(s_t, a_t, out_t, t0=t)
        pd_t           = model.state_trans_pd(s_t, a_t, out_t, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps_t, t=t)
        sim.outcomes[t] = out_t
        sim.actions[t]  = a_t
    return sim

# 2) Work‐income helper (unchanged)
def compute_work_income(states, outcomes, kappa):
    wages = states[..., 1]
    hours = outcomes[..., 2]
    return kappa[:, None] * wages * hours

# 3) Age‐bins
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64)]
age_labels = ["18–34", "35–49", "50–64"]

# 4) Pre‐draw & share shocks
torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw    = model_DL['Baseline'].draw_shocks(N).to(device)
model_DL['Baseline'].sim.shocks   = raw.clone()
model_DL['Experiment'].sim.shocks = raw.clone()

# 5) Collect %Δ
percent_changes = []
for (a_lo, a_hi) in age_bins:
    t_lo = a_lo - age_start

    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    sb = simulate_model(mb)
    Sb, Ob = sb.states.detach().cpu().numpy(), sb.outcomes.detach().cpu().numpy()
    kb      = mb.par.kappa.detach().cpu().numpy()
    WI_b    = compute_work_income(Sb, Ob, kb)

    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    Se, Oe = se.states.detach().cpu().numpy(), se.outcomes.detach().cpu().numpy()
    ke      = me.par.kappa.detach().cpu().numpy()
    WI_e    = compute_work_income(Se, Oe, ke)

    base = WI_b[t_lo:].ravel()
    exp  = WI_e[t_lo:].ravel()
    mask = base > 0
    pct  = (exp[mask] - base[mask]) / base[mask] * 100
    percent_changes.append(pct)

# 6) Plot with 1.5×IQR whiskers, median & mean, left‐side annotations
fig, ax = plt.subplots(figsize=(10,6))
x = np.arange(len(age_labels))

bp = ax.boxplot(
    percent_changes,
    positions=x,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

ax.axhline(0, linestyle="--", color="gray", linewidth=1)

meds  = np.array([np.nanmedian(a) for a in percent_changes])
means = np.array([np.nanmean(a)   for a in percent_changes])

# median diamonds
ax.scatter(x, meds, marker='D', s=60,
           facecolors='#2f4f4f', edgecolors='white',
           linewidths=1.2, zorder=4, label="Cohort Median")

# mean circles
ax.scatter(x, means, marker='o', s=50,
           facecolors='white', edgecolors='#e6550d',
           linewidths=1.5, zorder=5, label="Cohort Mean")

# annotation offsets
dx = -0.15  # leftwards
dy = 0.2

for i in range(len(age_labels)):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    m  = meds[i]
    mu = means[i]

    # min
    ax.text(x[i]+dx, lo, f"min {lo:.1f}%", ha="right", va="center", fontsize=8)
    # max
    ax.text(x[i]+dx, hi, f"max {hi:.1f}%", ha="right", va="center", fontsize=8)
    # median
    ax.text(x[i]+dx-0.2, m,  f"{m:.1f}%",    ha="right", va="center", fontsize=9, fontweight="bold")
    # mean (slightly below median)
    ax.text(x[i]+dx-0.2, mu-0.25, f"{mu:.1f}%", ha="right", va="center", fontsize=8, color="#e6550d")

ax.set_xticks(x)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Employment Income")
ax.grid(axis='y', linestyle=':', alpha=0.7)
ax.legend(frameon=False, loc='upper right')

plt.tight_layout()
#plt.savefig("mp_tight_employment_income.png", dpi=300)
plt.savefig("mp_soft_employment_income.png", dpi=300)
plt.show()


In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch


# 2) Income helpers
def compute_work_income(states, outcomes, kappa):
    wages = states[..., 1]
    hours = outcomes[..., 1]
    return kappa[:, None] * wages * hours

def compute_invest_income(states, outcomes):
    rtn_ill  = states[..., 5]
    rtn_liq  = states[..., 4]
    ill_new  = outcomes[..., 4]
    liq_new  = outcomes[..., 3]
    return (ill_new ) * (rtn_ill-1) + liq_new * (rtn_liq-1)

# 3) Age bins
age_start  = 18
#age_bins   = [(18, 34), (35, 49), (50, 64)]
#age_labels = ["18–34", "35–49", "50–64"]

#age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
age_bins   = [18, 35, 50, 65, 75, 85]
age_labels = ["18–", "35–", "50–", "65–", "75–", "85-"]

# 4) Pre-draw & share idio shocks
torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw    = model_DL['Baseline'].draw_shocks(N).to(device)
model_DL['Baseline'].sim.shocks   = raw.clone()
model_DL['Experiment'].sim.shocks = raw.clone()

# 5) Collect %Δ for work & investment
pct_work   = []
pct_invest = []

for a_lo in age_bins:
    t_lo = a_lo - age_start

    # baseline
    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    sb = simulate_model(mb)
    Sb, Ob = sb.states.detach().cpu().numpy(), sb.outcomes.detach().cpu().numpy()
    kb      = mb.par.kappa.detach().cpu().numpy()
    WI_b    = compute_work_income(Sb, Ob, kb)
    II_b    = compute_invest_income(Sb, Ob)

    # experiment
    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    Se, Oe = se.states.detach().cpu().numpy(), se.outcomes.detach().cpu().numpy()
    ke      = me.par.kappa.detach().cpu().numpy()
    WI_e    = compute_work_income(Se, Oe, ke)
    II_e    = compute_invest_income(Se, Oe)

    # percent changes (mask zero baseline)
    base_w = WI_b[t_lo:].ravel()
    exp_w  = WI_e[t_lo:].ravel()
    mask_w = base_w != 0
    pct_work.append((exp_w[mask_w] - base_w[mask_w]) / base_w[mask_w] * 100)

    base_i = II_b[t_lo:].ravel()
    exp_i  = II_e[t_lo:].ravel()
    mask_i = base_i != 0
    pct_invest.append((exp_i[mask_i] - base_i[mask_i]) / base_i[mask_i] * 100)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# assume pct_invest and age_labels are already defined
# pct_invest: list of 6 arrays, one per age bin
# age_labels: ["18–34","35–49","50–64","65–74","75–84","85+"]

# 1) set up a bigger figure
fig, ax = plt.subplots(figsize=(14, 8))

xpos = np.arange(len(age_labels))

# 2) Draw the boxplot
bp = ax.boxplot(
    pct_invest,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)

# 3) Color the boxes
for box in bp['boxes']:
    box.set_facecolor('#8dd3c7')
    box.set_alpha(0.7)

# 4) Zero line
ax.axhline(0, linestyle='--', color='gray', linewidth=1)

# 5) Compute medians & whisker‐restricted means
meds = np.array([np.nanmedian(arr) for arr in pct_invest])
means = []
for i, arr in enumerate(pct_invest):
    lo = bp['caps'][2*i].get_ydata()[0]
    hi = bp['caps'][2*i+1].get_ydata()[0]
    in_range = arr[(arr >= lo) & (arr <= hi)]
    means.append(np.nanmean(in_range))
means = np.array(means)

# 6) Scatter median & mean
ax.scatter(xpos, meds,
           marker='D', s=80,
           facecolors='#2f4f4f', edgecolors='white',
           linewidths=1.5, zorder=4,
           label='Cohort Median')
ax.scatter(xpos, means,
           marker='o', s=70,
           facecolors='white', edgecolors='#e6550d',
           linewidths=1.8, zorder=5,
           label='Cohort Mean')

# 7) Annotate min, max, median, mean to the right
for i, x in enumerate(xpos):
    lo = bp['caps'][2*i].get_ydata()[0]
    hi = bp['caps'][2*i+1].get_ydata()[0]
    med  = meds[i]
    mval = means[i]
    offset_x = 0.0

    # max
    ax.text(x+offset_x, hi,
            f"max {hi:.1f}%",
            ha='left', va='bottom',
            fontsize=10, fontweight='bold')
    # min
    ax.text(x+offset_x, lo-1.0,
            f"min {lo:.1f}%",
            ha='left', va='top',
            fontsize=10, fontweight='bold')
    # median
    ax.text(x+offset_x+0.3, med-2.5,
            f"{med:.1f}%",
            ha='left', va='bottom',
            fontsize=11, fontweight='bold', color='#2f4f4f')
    # mean
    ax.text(x+offset_x+0.3, mval+2.5,
            f"{mval:.1f}%",
            ha='left', va='top',
            fontsize=10, fontweight='bold', color='#e6550d')

# 8) Final styling
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha='right', fontsize=12)
ax.set_ylabel('Percent Change in Investment Income', fontsize=14)
#ax.set_title('Investment Income %Δ by Age Cohort', fontsize=16)
ax.grid(axis='y', linestyle=':', alpha=0.7)
ax.legend(frameon=False, fontsize=12, loc='upper left')

plt.tight_layout()
plt.savefig('investment_income_percent_change_large.png', dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# assume pct_work and age_labels are defined, and pct_work has at least 3 cohorts
data = pct_work[:3]
labels = age_labels[:3]
xpos = np.arange(len(labels))

# 1) Create a larger figure
fig, ax = plt.subplots(figsize=(14, 8))

# 2) Draw boxplot
bp = ax.boxplot(
    data,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)
for box in bp['boxes']:
    box.set_facecolor('#8dd3c7')
    box.set_alpha(0.7)

# 3) Zero reference line
ax.axhline(0, linestyle='--', color='gray', linewidth=1)

# 4) Compute medians & whisker‐restricted means
meds = np.array([np.nanmedian(arr) for arr in data])
means = []
for i, arr in enumerate(data):
    lo = bp['caps'][2*i].get_ydata()[0]
    hi = bp['caps'][2*i+1].get_ydata()[0]
    in_range = arr[(arr >= lo) & (arr <= hi)]
    means.append(np.nanmean(in_range))
means = np.array(means)

# 5) Plot median & mean markers
ax.scatter(xpos, meds,
           marker='D', s=80,
           facecolors='#2f4f4f', edgecolors='white',
           linewidths=1.5, zorder=4,
           label='Cohort Median')
ax.scatter(xpos, means,
           marker='o', s=70,
           facecolors='white', edgecolors='#e6550d',
           linewidths=1.8, zorder=5,
           label='Cohort Mean')

# 6) Annotate min, max, median, mean to the right
offset_x = -0.1
for i, x in enumerate(xpos):
    lo   = bp['caps'][2*i].get_ydata()[0]
    hi   = bp['caps'][2*i+1].get_ydata()[0]
    med  = meds[i]
    mval = means[i]

    ax.text(x+offset_x, hi,
            f"max {hi:.1f}%",
            ha='left', va='bottom',
            fontsize=10, fontweight='bold')
    ax.text(x+offset_x, lo,
            f"min {lo:.1f}%",
            ha='left', va='top',
            fontsize=10, fontweight='bold')
    ax.text(x+offset_x+0.1, med +0.1,
            f"{med:.1f}%",
            ha='left', va='bottom',
            fontsize=11, fontweight='bold', color='#2f4f4f')
    ax.text(x+offset_x+0.1, mval-0.1,
            f"{mval:.1f}%",
            ha='left', va='top',
            fontsize=10, fontweight='bold', color='#e6550d')

# 7) Final styling
ax.set_xticks(xpos)
ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=12)
ax.set_ylabel('Percent Change in Employment Income', fontsize=14)
#ax.set_title('Work Income %Δ by Age Cohort', fontsize=16)
ax.grid(axis='y', linestyle=':', alpha=0.7)
ax.legend(frameon=False, fontsize=12, loc='upper left')

plt.tight_layout()
plt.savefig('employment_income_percent_change_large.png', dpi=300)
plt.show()


In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch

# 3) Age bins
age_start  = 18
#age_bins   = [(18, 34), (35, 49), (50, 64)]
#age_labels = ["18–34", "35–49", "50–64"]

#age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
#age_labels = ["18–34", "35–49", "50–64", "65–74", "75–84", "85+"]

age_bins   = [18, 35, 50, 65, 75, 85]
age_labels = ["18–", "35–", "50–", "65–", "75–", "85-"]

# 4) Share the shocks between baseline and experiment
torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw    = model_DL['Baseline'].draw_shocks(N).to(device)
model_DL['Baseline'].sim.shocks   = raw.clone()
model_DL['Experiment'].sim.shocks = raw.clone()

# 5) Compute percent changes in cash‐on‐hand by age bin
pct_cash = []
for a_lo in age_bins:
    t_lo = a_lo - age_start

    # Baseline
    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    sb = simulate_model(mb)
    WI_b = sb.states[..., 2].detach().cpu().numpy()

    # Experiment
    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    WI_e = se.states[..., 2].detach().cpu().numpy()

    # Flatten from t_lo onward and drop zeros
    base = WI_b[t_lo:].ravel()
    exp  = WI_e[t_lo:].ravel()
    mask = base != 0
    pct_cash.append((exp[mask] - base[mask]) / base[mask] * 100)

# 6) Plot
fig, ax = plt.subplots(figsize=(10, 6))
xpos = np.arange(len(age_labels))

# 6a) Boxplot
bp = ax.boxplot(
    pct_cash,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)

# 6b) Colour boxes
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

# 6c) Zero line
ax.axhline(0, linestyle="--", color="gray", linewidth=1)

# 6d) Cohort medians & means
meds = np.array([np.nanmedian(arr) for arr in pct_cash])
means = []
for i, arr in enumerate(pct_cash):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    in_range = arr[(arr >= lo) & (arr <= hi)]
    means.append(np.nanmean(in_range))
means = np.array(means)

ax.scatter(
    xpos, meds,
    marker='D', s=60,
    facecolors='#2f4f4f', edgecolors='white',
    linewidths=1.2, zorder=4,
    label="Cohort Median"
)
ax.scatter(
    xpos, means,
    marker='o', s=50,
    facecolors='white', edgecolors='#e6550d',
    linewidths=1.5, zorder=5,
    label="Cohort Mean"
)

# 6e) Annotate medians & means (centred)
for i in range(len(age_labels)):
    ax.text(
        xpos[i]-0.1, meds[i] - 2.5,   # 1% above the median point
        f"{meds[i]:.1f}%",
        ha="center", va="bottom",
        fontsize=9, fontweight="bold", color="#2f4f4f"
    )
    ax.text(
        xpos[i] -0.1, means[i] + 2,  # 1% below the mean point
        f"{means[i]:.1f}%",
        ha="center", va="top",
        fontsize=8, fontweight = "bold", color="#e6550d"
    )
    # annotate whisker caps
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    ax.text(
        xpos[i], hi + 1.1,
        f"max {hi:.1f}%",
        ha="center", va="bottom",
        fontsize=8, fontweight = "bold", color="#333333"
    )
    ax.text(
        xpos[i], lo - 1.0,
        f"min {lo:.1f}%",
        ha="center", va="top",
        fontsize=8, fontweight = "bold", color="#333333"
    )

# 6f) Final formatting
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Cash-on-Hand")
#ax.set_title("Cash-on-Hand Shock Impacts by Age Cohort")
ax.grid(axis='y', linestyle=':', linewidth=0.8, alpha=0.7)
ax.legend(frameon=False, loc='upper left')

plt.tight_layout()
plt.savefig("mp_soft_cash_boxplot.png", dpi=300)
plt.show()


In [ ]:
# 6) Plot
fig, ax = plt.subplots(figsize=(10, 6))
xpos = np.arange(len(age_labels))

# 6a) Boxplot
bp = ax.boxplot(
    pct_cash,
    positions=xpos,
    widths=0.6,
    notch=True,
    showfliers=False,
    whis=1.5,
    patch_artist=True,
    medianprops=dict(color="#2f4f4f", linewidth=1.5),
    whiskerprops=dict(color="#555555", linewidth=1),
    capprops=dict(color="#555555", linewidth=1),
)

# 6b) Colour boxes
for box in bp["boxes"]:
    box.set_facecolor("#8dd3c7")
    box.set_alpha(0.7)

# 6c) Zero line
ax.axhline(0, linestyle="--", color="gray", linewidth=1)

# 6d) Cohort medians & means
meds = np.array([np.nanmedian(arr) for arr in pct_cash])
means = []
for i, arr in enumerate(pct_cash):
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    in_range = arr[(arr >= lo) & (arr <= hi)]
    means.append(np.nanmean(in_range))
means = np.array(means)

ax.scatter(
    xpos, meds,
    marker='D', s=60,
    facecolors='#2f4f4f', edgecolors='white',
    linewidths=1.2, zorder=4,
    label="Cohort Median"
)
ax.scatter(
    xpos, means,
    marker='o', s=50,
    facecolors='white', edgecolors='#e6550d',
    linewidths=1.5, zorder=5,
    label="Cohort Mean"
)

# 6e) Annotate medians & means (centred)
for i in range(len(age_labels)):
    ax.text(
        xpos[i]-0.1, meds[i] - 5.3,   # 1% above the median point
        f"{meds[i]:.1f}%",
        ha="center", va="bottom",
        fontsize=9, fontweight="bold", color="#2f4f4f"
    )
    ax.text(
        xpos[i] -0.1, means[i] + 5.5,  # 1% below the mean point
        f"{means[i]:.1f}%",
        ha="center", va="top",
        fontsize=8, fontweight = "bold", color="#e6550d"
    )
    # annotate whisker caps
    lo = bp["caps"][2*i].get_ydata()[0]
    hi = bp["caps"][2*i+1].get_ydata()[0]
    ax.text(
        xpos[i], hi + 1.1,
        f"max {hi:.1f}%",
        ha="center", va="bottom",
        fontsize=8, fontweight = "bold", color="#333333"
    )
    ax.text(
        xpos[i], lo - 1.0,
        f"min {lo:.1f}%",
        ha="center", va="top",
        fontsize=8, fontweight = "bold", color="#333333"
    )

# 6f) Final formatting
ax.set_xticks(xpos)
ax.set_xticklabels(age_labels, rotation=45, ha="right")
ax.set_ylabel("Percent Change in Cash-on-Hand")
#ax.set_title("Cash-on-Hand Shock Impacts by Age Cohort")
ax.grid(axis='y', linestyle=':', linewidth=0.8, alpha=0.7)
ax.legend(frameon=False, loc='upper left')

plt.tight_layout()
plt.savefig("mp_soft_cash_boxplot.png", dpi=300)
plt.show()


In [ ]:

#misc
def simulate_model(model):
    par = model.par
    sim = model.sim

    # initial conditions
    sim.states[0] = model.draw_initial_states(sim.N)
    sim.shocks    = model.draw_shocks(sim.N).to(sim.states.device)

    # lifecycle
    for t in range(par.T - 1):
        s_t   = sim.states[t]
        eps_t = sim.shocks[t]

        a_t        = model.eval_policy(model.policy_NN, s_t.unsqueeze(0)).squeeze(0)
        out_t      = model.outcomes(s_t, a_t, t0=t, t=t)
        sim.reward[t] = model.reward(s_t, a_t, out_t, t0=t)

        pd_t           = model.state_trans_pd(s_t, a_t, out_t, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps_t, t=t)

        sim.outcomes[t] = out_t
        sim.actions[t]  = a_t

    return sim
# --- 1) Work‐income helper ---
def compute_work_income(states, outcomes, kappa):
    """
    states:   (T, N, n_states)    where state index 1 is wage
    outcomes: (T, N, n_outcomes)   where outcome index 2 is labor hours share
    kappa:    (T,) array of labor efficiency
    Returns:  (T, N) work-related income = kappa[t] * wage * hours_share
    """
    wages      = states[..., 1]      # wage
    hours      = outcomes[..., 2]    # labor hours share
    # broadcast kappa across agents
    return kappa[:, None] * wages * hours

# --- 3) Age‐bin definitions ---
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
age_labels = ["18–34", "35–49", "50–64", "65–74", "75–84", "85+"]

# --- 4) Loop over bins, collect pooled work‐income lists ---
bins_hi = []  # Baseline
bins_lo = []  # Experiment

for (a_lo, a_hi) in age_bins:
    # map age‐range to t‐range
    t_lo  = a_lo - age_start
    t_hi  = min(a_hi - age_start, model_DL['Baseline'].par.T - 1)
    t_mid = (t_lo + t_hi) // 2
    periods = [t_lo, t_mid, t_hi]

    # --- Baseline run (no shock) ---
    model_DL['Baseline'].par.shock_periods   = []
    sim_b = simulate_model(model_DL['Baseline'])
    sh_b, oh_b = sim_b.states.detach().cpu().numpy(), sim_b.outcomes.detach().cpu().numpy()
    kappa_b    = model_DL['Baseline'].par.kappa.detach().cpu().numpy()
    wi_b       = compute_work_income(sh_b, oh_b, kappa_b)

    # --- Experiment run (shock full bin) ---
    model_DL['Experiment'].par.shock_periods = periods
    sim_e = simulate_model(model_DL['Experiment'])
    sh_e, oh_e = sim_e.states.detach().cpu().numpy(), sim_e.outcomes.detach().cpu().numpy()
    kappa_e    = model_DL['Experiment'].par.kappa.detach().cpu().numpy()
    wi_e       = compute_work_income(sh_e, oh_e, kappa_e)

    # pool all ages in bin: flatten work‐income over those periods and all sims
    pooled_b = wi_b[periods, :].ravel()
    pooled_e = wi_e[periods, :].ravel()
    bins_hi.append(pooled_b)
    bins_lo.append(pooled_e)

# --- 5) Plot with the desired styling ---
labels = age_labels
x      = np.arange(len(labels))
pos_hi = x - 0.15
pos_lo = x + 0.15
clr_hi = "#ff9999"
clr_lo = "#66b3ff"

common_kw = dict(
    notch=True,
    showfliers=False,
    whis=1.5,
    medianprops=dict(color="black", linewidth=1.2),
)

fig, ax = plt.subplots(figsize=(10, 6))

bp_hi = ax.boxplot(
    bins_hi, positions=pos_hi, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_hi, alpha=0.6, linewidth=1),
    **common_kw
)
bp_lo = ax.boxplot(
    bins_lo, positions=pos_lo, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_lo, alpha=0.6, linewidth=1),
    **common_kw
)

ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel("Work‐related income")
ax.legend(
    [bp_hi["boxes"][0], bp_lo["boxes"][0]],
    ["Baseline", "Experiment"],
    loc="upper left",
    frameon=False
)

# annotate one box, e.g. "50–64"
i = labels.index("50–64")
data = bins_hi[i]
q1, med, q3 = np.percentile(data, [25, 50, 75])
iqr = q3 - q1
whisk_low  = np.min(data[data >= q1 - 1.5 * iqr])
whisk_high = np.max(data[data <= q3 + 1.5 * iqr])
mean_val   = np.mean(data)

stat_text = (
    "Age 50–64, Baseline\n"
    f"Min    = {whisk_low:.0f}\n"
    f"Median = {med:.0f}\n"
    f"Mean   = {mean_val:.0f}\n"
    f"Max    = {whisk_high:.0f}"
)

ax.annotate(
    stat_text,
    xy=(pos_hi[i], med),
    xytext=(0.02, 0.75),
    textcoords="axes fraction",
    arrowprops=dict(arrowstyle="->", lw=1),
    va="center", ha="left",
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.4",
              facecolor="white", edgecolor="gray", linewidth=1)
)

plt.tight_layout()
plt.savefig("boxplot_work_income_baseline_experiment.png")
plt.show()
#plt.savefig("boxplot_work_income_baseline_experiment_SOFT.png")



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1) simulate_model helper (mutates model.sim and returns it) ---
def simulate_model(model):
    par = model.par
    sim = model.sim

    # initial conditions
    sim.states[0] = model.draw_initial_states(sim.N)
    sim.shocks    = model.draw_shocks(sim.N).to(sim.states.device)

    # lifecycle
    for t in range(par.T - 1):
        s_t   = sim.states[t]
        eps_t = sim.shocks[t]

        a_t          = model.eval_policy(model.policy_NN, s_t.unsqueeze(0)).squeeze(0)
        out_t        = model.outcomes(s_t, a_t, t0=t, t=t)
        sim.reward[t] = model.reward(s_t, a_t, out_t, t0=t)

        pd_t           = model.state_trans_pd(s_t, a_t, out_t, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps_t, t=t)

        sim.outcomes[t] = out_t
        sim.actions[t]  = a_t

    return sim

# --- 2) Investment‐income helper ---
def compute_invest_income(states, outcomes):
    """
    states:   (T, N, n_states)
      idx0 = illiquid assets carried over
      idx5 = return on illiquid
      idx4 = return on liquid
    outcomes: (T, N, n_outcomes)
      idx4 = new illiquid assets
      idx3 = new liquid assets
    """
    ill_prev = states[..., 0]
    rtn_ill  = states[..., 5]
    rtn_liq  = states[..., 4]
    ill_new  = outcomes[..., 4]
    liq_new  = outcomes[..., 3]
    return (ill_prev + ill_new) * rtn_ill + liq_new * rtn_liq

# --- 3) Age‐bin definitions ---
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64), (65, 74), (75, 84), (85, 200)]
age_labels = ["18–34", "35–49", "50–64", "65–74", "75–84", "85+"]

# --- 4) Loop over bins, collect pooled investment‐income lists ---
bins_hi = []  # Baseline (Patient β=0.99)
bins_lo = []  # Experiment (Impatient β=0.95)

for (a_lo, a_hi) in age_bins:
    t_lo  = a_lo - age_start
    t_hi  = min(a_hi - age_start, model_DL['Baseline'].par.T - 1)
    t_mid = (t_lo + t_hi) // 2
    periods = [t_lo, t_mid, t_hi]

    # Baseline run
    model_DL['Baseline'].par.shock_periods = periods
    sim_b = simulate_model(model_DL['Baseline'])
    sh_b, oh_b = sim_b.states.detach().cpu().numpy(), sim_b.outcomes.detach().cpu().numpy()
    inv_b      = compute_invest_income(sh_b, oh_b)

    # Experiment run
    model_DL['Experiment'].par.shock_periods = periods
    sim_e = simulate_model(model_DL['Experiment'])
    sh_e, oh_e = sim_e.states.detach().cpu().numpy(), sim_e.outcomes.detach().cpu().numpy()
    inv_e      = compute_invest_income(sh_e, oh_e)

    # pool across the bin
    bins_hi.append(inv_b[periods, :].ravel())
    bins_lo.append(inv_e[periods, :].ravel())

# --- 5) Plot with consistent box‐plot style ---
labels = age_labels
x      = np.arange(len(labels))
pos_hi = x - 0.15
pos_lo = x + 0.15
clr_hi = "#ff9999"
clr_lo = "#66b3ff"

common_kw = dict(
    notch=True,
    showfliers=False,
    whis=1.5,
    medianprops=dict(color="black", linewidth=1.2),
)

fig, ax = plt.subplots(figsize=(10, 6))

bp_hi = ax.boxplot(
    bins_hi, positions=pos_hi, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_hi, alpha=0.6, linewidth=1),
    **common_kw
)
bp_lo = ax.boxplot(
    bins_lo, positions=pos_lo, widths=0.28, patch_artist=True,
    boxprops=dict(facecolor=clr_lo, alpha=0.6, linewidth=1),
    **common_kw
)

ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel("Investment income")
ax.legend(
    [bp_hi["boxes"][0], bp_lo["boxes"][0]],
    ["Baseline", "Experiment"],
    loc="upper left",
    frameon=False
)

# Annotate stats for "50–64", Baseline
i = labels.index("50–64")
data = bins_hi[i]
q1, med, q3 = np.percentile(data, [25, 50, 75])
iqr = q3 - q1
whisk_low  = np.min(data[data >= q1 - 1.5 * iqr])
whisk_high = np.max(data[data <= q3 + 1.5 * iqr])
mean_val   = data.mean()

stat_text = (
    "Age 50–64, Baseline\n"
    f"Min    = {whisk_low:.0f}\n"
    f"Median = {med:.0f}\n"
    f"Mean   = {mean_val:.0f}\n"
    f"Max    = {whisk_high:.0f}"
)

ax.annotate(
    stat_text,
    xy=(pos_hi[i], med),
    xytext=(0.05, 0.75),
    textcoords="axes fraction",
    arrowprops=dict(arrowstyle="->", lw=1, color="gray"),
    va="center", ha="left",
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.4",
              facecolor="white", edgecolor="gray", linewidth=1)
)

plt.tight_layout()
plt.show()
fig.savefig("boxplot_investment_income.png")


In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch

# --- 1) simulate_model helper (unchanged) ---
def simulate_model(model):
    par = model.par
    sim = model.sim
    sim.states[0] = model.draw_initial_states(sim.N)
    # assume sim.shocks already set
    for t in range(par.T - 1):
        s_t   = sim.states[t]
        eps_t = sim.shocks[t]
        a_t   = model.eval_policy(model.policy_NN, s_t.unsqueeze(0)).squeeze(0)
        out_t = model.outcomes(s_t, a_t, t0=t, t=t)
        sim.reward[t] = model.reward(s_t, a_t, out_t, t0=t)
        pd_t = model.state_trans_pd(s_t, a_t, out_t, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps_t, t=t)
        sim.outcomes[t] = out_t
        sim.actions[t]  = a_t
    return sim

# --- 2) Income helpers (work + invest) ---
def compute_work_income(states, outcomes, kappa):
    wages = states[..., 1]
    hours = outcomes[..., 2]
    return kappa[:, None] * wages * hours

def compute_invest_income(states, outcomes):
    ill_prev = states[..., 0]
    rtn_ill  = states[..., 5]
    rtn_liq  = states[..., 4]
    ill_new  = outcomes[..., 4]
    liq_new  = outcomes[..., 3]
    return (ill_prev + ill_new) * rtn_ill + liq_new * rtn_liq

# --- 3) Lorenz helper (unchanged) ---
def lorenz_curve(x):
    x = np.clip(x, 0, None)
    sorted_x = np.sort(x)
    cumx = np.concatenate(([0], np.cumsum(sorted_x)))
    p = np.linspace(0, 1, len(cumx))
    w = cumx / cumx[-1]
    return p, w

# --- 4) Age bins + idio shocks ---
age_start  = 18
age_bins   = [(18, 34), (35, 49), (50, 64)]
torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw_idio = model_DL['Baseline'].draw_shocks(N).to(device)

bins_hi = []
bins_lo = []

for (a_lo, a_hi) in age_bins:
    t_lo = a_lo - age_start
    t_hi = min(a_hi - age_start, model_DL['Baseline'].par.T - 1)
    periods = list(range(t_lo, t_hi + 1))

    # Baseline
    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    mb.sim.shocks = raw_idio.clone()
    sb = simulate_model(mb)
    Sb, Ob = sb.states.detach().cpu().numpy(), sb.outcomes.detach().cpu().numpy()
    kb      = mb.par.kappa.detach().cpu().numpy()
    wi_b = compute_work_income(Sb, Ob, kb)
    ii_b = compute_invest_income(Sb, Ob)
    tot_b = wi_b + ii_b

    # Experiment
    me = copy.deepcopy(model_DL['Experiment'])
    shocks_e = raw_idio.clone()
    shocks_e[t_lo, :, 1] += me.par.R_shock
    me.sim.shocks       = shocks_e
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    Se, Oe = se.states.detach().cpu().numpy(), se.outcomes.detach().cpu().numpy()
    ke      = me.par.kappa.detach().cpu().numpy()
    wi_e = compute_work_income(Se, Oe, ke)
    ii_e = compute_invest_income(Se, Oe)
    tot_e = wi_e + ii_e

    bins_hi.append(tot_b[periods, :].ravel())
    bins_lo.append(tot_e[periods, :].ravel())

# --- 5) Pool full distribution ---
all_hi = np.concatenate(bins_hi)
all_lo = np.concatenate(bins_lo)

p_hi, w_hi = lorenz_curve(all_hi)
p_lo, w_lo = lorenz_curve(all_lo)

gap_hi = p_hi - w_hi
gap_lo = p_lo - w_lo
i_hi = np.argmax(gap_hi)
i_lo = np.argmax(gap_lo)

pmax_hi, gapmax_hi = p_hi[i_hi], gap_hi[i_hi]
pmax_lo, gapmax_lo = p_lo[i_lo], gap_lo[i_lo]
# --- 6) Plot Lorenz curves with labelled max-gap annotations ---
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(p_hi, w_hi, lw=2, label="Baseline")
ax.plot(p_lo, w_lo, lw=2, label="Experiment")
ax.plot([0,1], [0,1], "k--", alpha=0.6, label="Line of equality")

# Baseline max-gap
ax.annotate(
    f"Baseline max gap {gapmax_hi*100:.1f}% at {pmax_hi*100:.1f}%",
    xy=(pmax_hi, w_hi[i_hi]),
    xytext=(0.05, 0.65),
    textcoords="axes fraction",
    arrowprops=dict(arrowstyle="->", lw=1),
    bbox=dict(facecolor="white", edgecolor="gray", pad=0.3),
)

# Experiment max-gap
ax.annotate(
    f"Experiment max gap {gapmax_lo*100:.1f}% at {pmax_lo*100:.1f}%",
    xy=(pmax_lo, w_lo[i_lo]),
    xytext=(0.55, 0.30),
    textcoords="axes fraction",
    arrowprops=dict(arrowstyle="->", lw=1),
    bbox=dict(facecolor="white", edgecolor="gray", pad=0.3),
)

ax.set_xlabel("Cumulative share of population")
ax.set_ylabel("Cumulative share of total income")
ax.legend(loc="lower right")
ax.grid(alpha=0.3)

fig.tight_layout()
fig.savefig("lorenz_total_income_baseline_experiment_labeled.png", dpi=300, bbox_inches="tight")
plt.show()



In [ ]:
import numpy as np
import pandas as pd
import copy
import torch

# 2) Gini helper
def gini_coefficient(x: np.ndarray) -> float:
    """
    Compute Gini coefficient for non‐negative array x.
    Returns 0.0 if x is all zero or empty.
    """
    x = x.flatten()
    x = x[x >= 0]
    if x.size == 0 or x.sum() == 0:
        return 0.0
    x = np.sort(x)
    n = x.size
    cumx = np.cumsum(x)
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n


# 3) compute helpers
def compute_net_wealth(S, O):
    return O[...,4] + O[...,3] - S[...,6] - O[...,5]

def compute_work_income(S, O, kappa):
    return kappa[:,None] * S[...,1] * O[...,1]

def compute_invest_income(S, O):
    return O[...,4] * (S[...,5]-1) + O[...,3] * (S[...,4]-1)

# 4) age bins
age_start = 18
age_bins   = [18, 35, 50, 65, 75, 85]
age_labels = ["18–", "35–", "50–", "65–", "75–", "85-"]

# 5) Pre-draw everything once
torch.manual_seed(1337)
N      = model_DL['Baseline'].sim.N
device = model_DL['Baseline'].sim.states.device
raw    = model_DL['Baseline'].draw_shocks(N).to(device)
model_DL['Baseline'].sim.shocks   = raw.clone()
model_DL['Experiment'].sim.shocks = raw.clone()

# 6) Build rows
rows = []
for a_lo, label in zip(age_bins, age_labels):
    t_lo = a_lo - age_start
    sl   = slice(t_lo, None)  # From t_lo to end of horizon

    # -- Baseline --
    mb = copy.deepcopy(model_DL['Baseline'])
    mb.par.shock_periods = []
    sb = simulate_model(mb)
    S_b = sb.states.detach().cpu().numpy()
    O_b = sb.outcomes.detach().cpu().numpy()
    k_b = mb.par.kappa.detach().cpu().numpy()

    # -- Experiment --
    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    S_e = se.states.detach().cpu().numpy()
    O_e = se.outcomes.detach().cpu().numpy()
    k_e = me.par.kappa.detach().cpu().numpy()

    for model_key, (S,O,kappa) in [
        ("Baseline",(S_b,O_b,k_b)),
        ("Experiment",(S_e,O_e,k_e))
    ]:
        cons = O[...,0][sl,:].ravel()
        nw   = compute_net_wealth(S,O)[sl,:].ravel()
        inc  = (compute_work_income(S,O,kappa) + compute_invest_income(S,O))[sl,:].ravel()
        cash = S[...,2][sl,:].ravel()

        rows.append({
            'Age bin': label,
            'Model':   model_key,
            'Gini_consumption':  gini_coefficient(cons)*100,
            'Gini_net_wealth':   gini_coefficient(nw)*100,
            'Gini_total_income': gini_coefficient(inc)*100,
            'Gini_cash_on_hand': gini_coefficient(cash)*100
        })

df = pd.DataFrame(rows).pivot(index='Age bin', columns='Model')
df


In [ ]:
import pandas as pd

# assuming `df` is your pivoted DataFrame from before, with columns like
# ('Gini_consumption','Baseline'), ('Gini_consumption','Experiment'), etc.

# extract the baseline and experiment blocks
gini_base = df.xs('Baseline', axis=1, level=1)
gini_exp  = df.xs('Experiment', axis=1, level=1)

# compute percent change
gini_pct_change = (gini_exp - gini_base)  #/ gini_base * 100

# round to 3 decimals
gini_pct_change = gini_pct_change.round(3)

print(gini_pct_change)


In [ ]:
import numpy as np
import pandas as pd
import copy
import torch

def simulate_model(model):
    par = model.par
    sim = model.sim
    sim.states[0] = model.draw_initial_states(sim.N)
    sim.shocks    = model.draw_shocks(sim.N).to(sim.states.device)
    for t in range(par.T-1):
        s = sim.states[t]
        eps = sim.shocks[t]
        a = model.eval_policy(model.policy_NN, s.unsqueeze(0)).squeeze(0)
        out = model.outcomes(s, a, t0=t, t=t)
        sim.reward[t]   = model.reward(s, a, out, t0=t)
        pd_t            = model.state_trans_pd(s, a, out, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps, t=t)
        sim.outcomes[t] = out
        sim.actions[t]  = a
    return sim

def compute_net_wealth(states, outcomes):
    ill  = outcomes[..., 4]
    liq  = outcomes[..., 3]
    debt = states[..., 6]
    nd   = outcomes[..., 5]
    return ill + liq - debt - nd

def compute_consumption(outcomes):
    return outcomes[..., 0]

def compute_total_income(states, outcomes, kappa):
    work   = kappa[:,None] * states[...,1] * outcomes[...,1]
    invest = (outcomes[...,4]) * states[...,5] + outcomes[...,3] * states[...,4]
    return work + invest

def gini_coefficient(x):
    x = x.flatten()
    x = x[x >= 0]
    if len(x) < 2:
        return np.nan
    x = np.sort(x)
    n = len(x)
    cumx = np.cumsum(x)
    return (n + 1 - 2*np.sum(cumx)/cumx[-1]) / n

age_start  = 18
age_bins   = [(18,34),(35,49),(50,64),(65,74),(75,84),(85,200)]
age_labels = ["18–34","35–49","50–64","65–74","75–84","85+"]

rows = []
torch.manual_seed(1337)

for (a_lo,a_hi), label in zip(age_bins, age_labels):
    t_lo = a_lo - age_start
    t_hi = min(a_hi - age_start, model_DL['Baseline'].par.T - 1)
    sl   = slice(t_lo, t_hi+1)

    # baseline
    mb = copy.deepcopy(model_DL['Baseline'])
    sb = simulate_model(mb)
    S_b = sb.states.detach().cpu().numpy()
    O_b = sb.outcomes.detach().cpu().numpy()
    k_b = mb.par.kappa.detach().cpu().numpy()

    # experiment
    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    S_e = se.states.detach().cpu().numpy()
    O_e = se.outcomes.detach().cpu().numpy()
    k_e = me.par.kappa.detach().cpu().numpy()

    # slice cohorts
    c_b   = compute_consumption(O_b)[sl,:]
    c_e   = compute_consumption(O_e)[sl,:]
    nw_b  = compute_net_wealth(S_b, O_b)[sl,:]
    nw_e  = compute_net_wealth(S_e, O_e)[sl,:]
    inc_b = compute_total_income(S_b, O_b, k_b)[sl,:]
    inc_e = compute_total_income(S_e, O_e, k_e)[sl,:]

    # compute Gini levels
    Gc_b = gini_coefficient(c_b)
    Gc_e = gini_coefficient(c_e)
    Gnw_b = gini_coefficient(nw_b)
    Gnw_e = gini_coefficient(nw_e)
    Gi_b = gini_coefficient(inc_b)
    Gi_e = gini_coefficient(inc_e)

    # percentage changes relative to baseline
    pctC = (Gc_e - Gc_b) / Gc_b * 100 if Gc_b else np.nan
    pctNW = (Gnw_e - Gnw_b) / Gnw_b * 100 if Gnw_b else np.nan
    pctI = (Gi_e - Gi_b) / Gi_b * 100 if Gi_b else np.nan

    rows.append({
        'Age bin':                label,
        'GiniC_base':             Gc_b,
        'GiniC_experiment':       Gc_e,
        '%ΔGiniC':                pctC,
        'GiniNW_base':            Gnw_b,
        'GiniNW_experiment':      Gnw_e,
        '%ΔGiniNW':               pctNW,
        'GiniY_base':             Gi_b,
        'GiniY_experiment':       Gi_e,
        '%ΔGiniY':                pctI,
    })

df = pd.DataFrame(rows).set_index('Age bin')
print(df[['%ΔGiniC','%ΔGiniNW','%ΔGiniY']])


In [ ]:
df = pd.DataFrame(rows).set_index('Age bin')
print(df[['%ΔGiniC','%ΔGiniNW','%ΔGiniY']].round(2))


In [ ]:
import numpy as np
import pandas as pd
import copy
import torch

def simulate_model(model):
    par = model.par
    sim = model.sim

    # draw fresh initial states & shocks
    sim.states[0] = model.draw_initial_states(sim.N)
    sim.shocks    = model.draw_shocks(sim.N).to(sim.states.device)

    for t in range(par.T - 1):
        s_t   = sim.states[t]
        eps_t = sim.shocks[t]
        a_t   = model.eval_policy(model.policy_NN, s_t.unsqueeze(0)).squeeze(0)
        out_t = model.outcomes(s_t, a_t, t0=t, t=t)

        sim.reward[t]   = model.reward(s_t, a_t, out_t, t0=t)
        pd_t            = model.state_trans_pd(s_t, a_t, out_t, t0=t)
        sim.states[t+1] = model.state_trans(pd_t, eps_t, t=t)
        sim.outcomes[t] = out_t
        sim.actions[t]  = a_t

    return sim

def compute_net_wealth(states, outcomes):
    ill  = outcomes[..., 4]
    liq  = outcomes[..., 3]
    debt = states[..., 6]
    nd   = outcomes[..., 5]
    return ill + liq - debt - nd

age_start  = 18
age_bins   = [(18,34),(35,49),(50,64),(65,74),(75,84),(85,200)]
age_labels = ["18–34","35–49","50–64","65–74","75–84","85+"]

percentiles = [1,5,10,25,50,75,90,95,99]

# collect percent-change vectors for each cohort
percent_changes = []
for (a_lo, a_hi) in age_bins:
    t_lo = a_lo - age_start

    mb = copy.deepcopy(model_DL['Baseline'])
    sb = simulate_model(mb)
    sb_s, sb_o = sb.states.detach().cpu().numpy(), sb.outcomes.detach().cpu().numpy()
    nw_b = compute_net_wealth(sb_s, sb_o)

    me = copy.deepcopy(model_DL['Experiment'])
    me.par.shock_periods = [t_lo]
    se = simulate_model(me)
    se_s, se_o = se.states.detach().cpu().numpy(), se.outcomes.detach().cpu().numpy()
    nw_e = compute_net_wealth(se_s, se_o)

    pc = (nw_e[t_lo:,:] - nw_b[t_lo:,:]) / nw_b[t_lo:,:] * 100
    percent_changes.append(pc.ravel())

# build whisker‐filtered percentile table matching matplotlib.boxplot
rows = []
for pc, label in zip(percent_changes, age_labels):
    # drop non-finite
    pc = pc[np.isfinite(pc)]
    # compute Q1, Q3 with midpoint interpolation
    #q1, q3 = np.nanpercentile(pc, [25, 75], method="midpoint")
    #iqr    = q3 - q1
    #lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    # filter to whisker range
    #pc_in = pc[(pc >= lo) & (pc <= hi)]
    # compute percentiles
    vals = np.nanpercentile(pc , percentiles, method="midpoint")
    for p, v in zip(percentiles, vals):
        rows.append({'Age bin': label, 'Percentile': p, 'NetWealth%Gain': v})

df = pd.DataFrame(rows)
df_percentiles = df.pivot(index='Percentile', columns='Age bin', values='NetWealth%Gain')
print(df_percentiles)


In [ ]:
print(df.round(3))

In [ ]:
rows = []
for pc, label in zip(percent_changes, age_labels):
    vals = np.nanpercentile(pc, percentiles)
    for p, v in zip(percentiles, vals):
        rows.append({'Age bin': label, 'Percentile': p, 'NetWealth%Gain': v})

df = pd.DataFrame(rows).round(1)
table = df.pivot(index='Percentile', columns='Age bin', values='NetWealth%Gain')
print(table)

In [ ]:
gini_pct_change.to_csv("gini_coefficients_soft_pct_chg.csv")
